In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from functools import reduce

In [2]:
spark = SparkSession.builder.appName("MeuApp").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/31 18:07:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/31 18:07:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:

df_classe = spark.read.option("delimiter", ",").option("header", True).csv("classe.csv")

df_classe.show(truncate=False)

+-----------+------+-------+
|classe     |cd_cep|posicao|
+-----------+------+-------+
|CLASSE 21_5|21060 |5      |
|CLASSE 98_5|98495 |5      |
|CLASSE 38_5|38321 |5      |
|CLASSE 68_5|68789 |5      |
|CLASSE 72_5|71236 |5      |
|CLASSE 24_5|24147 |5      |
|CLASSE 07_5|07663 |5      |
|CLASSE 05_5|05886 |5      |
|CLASSE 77_5|77370 |5      |
|CLASSE 98_5|98574 |5      |
|CLASSE 04_5|04907 |5      |
|CLASSE 19_5|19892 |5      |
|CLASSE 33_5|33513 |5      |
|CLASSE 07_2|07    |2      |
|CLASSE 05_2|05    |2      |
|CLASSE 77_2|77    |2      |
|CLASSE 33_2|33    |2      |
|CLASSE 0_1 |0     |1      |
+-----------+------+-------+



In [4]:

df_fake = spark.read.option("delimiter", ",").option("header", True).csv("data.csv")\
.withColumn("data_nascimento", F.to_date(F.col("data_nascimento")))\
.withColumn("cep", F.regexp_replace(F.col("cep"), "-", ""))\
.withColumn("p_1", F.substring(F.col("cep"),  1, 1))\
.withColumn("p_2", F.substring(F.col("cep"),  1, 2))\
.withColumn("p_3", F.substring(F.col("cep"),  1, 3))\
.withColumn("p_4", F.substring(F.col("cep"),  1, 4))\
.withColumn("p_5", F.substring(F.col("cep"),  1, 5))\

df_fake.createOrReplaceTempView("tb_clientes")

df_fake.show(truncate=False)

+------+------------------------+----------------------------------+---------------+--------+---+---+---+----+-----+
|id    |nome                    |email                             |data_nascimento|cep     |p_1|p_2|p_3|p_4 |p_5  |
+------+------------------------+----------------------------------+---------------+--------+---+---+---+----+-----+
|605755|Catarina Rios           |costelasara@example.com           |1941-07-19     |72828003|7  |72 |728|7282|72828|
|453942|João Gabriel Nunes      |arthurdas-neves@example.org       |2004-12-01     |05055581|0  |05 |050|0505|05055|
|264157|Sofia Araújo            |bda-costa@example.com             |1910-11-02     |30758720|3  |30 |307|3075|30758|
|284311|Alexia Fogaça           |machadonina@example.org           |1929-01-29     |28840655|2  |28 |288|2884|28840|
|640344|Srta. Alexia Fonseca    |joao-guilhermesousa@example.org   |2002-11-26     |85644776|8  |85 |856|8564|85644|
|566589|Esther Correia          |vargasmateus@example.com       

In [5]:
lista_posicoes = [5, 4, 3, 2, 1]
series_append = []

for posicao in lista_posicoes:

    df_prep = None
    df_prep = spark.sql(f"""
                        SELECT
                            id,
                            nome, 
                            email, 
                            data_nascimento, 
                            cep, 
                            p_{posicao} as codigo_regiao
                        FROM
                            tb_clientes
                        """)
    df_prep = df_prep.withColumn("index_pos", F.lit(posicao))
    series_append.append(df_prep)

df = reduce(DataFrame.unionAll, series_append)

df.show(truncate=False, n=500)
print("Rows: ", df.count())

+------+-------------------------+----------------------------------+---------------+--------+-------------+---------+
|id    |nome                     |email                             |data_nascimento|cep     |codigo_regiao|index_pos|
+------+-------------------------+----------------------------------+---------------+--------+-------------+---------+
|605755|Catarina Rios            |costelasara@example.com           |1941-07-19     |72828003|72828        |5        |
|453942|João Gabriel Nunes       |arthurdas-neves@example.org       |2004-12-01     |05055581|05055        |5        |
|264157|Sofia Araújo             |bda-costa@example.com             |1910-11-02     |30758720|30758        |5        |
|284311|Alexia Fogaça            |machadonina@example.org           |1929-01-29     |28840655|28840        |5        |
|640344|Srta. Alexia Fonseca     |joao-guilhermesousa@example.org   |2002-11-26     |85644776|85644        |5        |
|566589|Esther Correia           |vargasmateus@e